<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 3: Coding Attention Mechanisms

Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.4.0


- This chapter covers attention mechanisms, the engine of LLMs:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 The problem with modeling long sequences

- No code in this section
- Translating a text word by word isn't feasible due to the differences in grammatical structures between the source and target languages:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

 Prior to the introduction of transformer models, encoder-decoder RNNs were commonly used for machine translation tasks
- In this setup, the encoder processes a sequence of tokens from the source language, using a hidden state—a kind of intermediate layer within the neural network—to generate a condensed representation of the entire input sequence:-

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 Capturing data dependencies with attention mechanisms

- No code in this section
- Through an attention mechanism, the text-generating decoder segment of the network is capable of selectively accessing all input tokens, implying that certain input tokens hold more significance than others in the generation of a specific output token:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- Self-attention in transformers is a technique designed to enhance input representations by enabling each position in a sequence to engage with and determine the relevance of every other position within the same sequence

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 Attending to different parts of the input with self-attention

### 3.3.1 A simple self-attention mechanism without trainable weights

本节解释了一种非常简化的自注意力机制，该机制不包含任何可训练的权重。
这纯粹是用于说明目的，而不是用于变换器（transformers）中的注意力机制。
下一节，即3.3.2节，将扩展这种简单的注意力机制以实现真正的自注意力机制。

假设我们有一个输入序列$x^{(1)}$到$x^{(T)}$：
输入是文本（例如，像“Your journey starts with one step”这样已经转换为如第2章所述的词嵌入的句子）。
例如，$x^{(1)}$是表示单词“Your”的d维向量，依此类推。

目标：为输入序列中的每个输入元素$x^{(i)}$计算上下文向量$z^{(i)}$（从$x^{(1)}$到$x^{(T)}$，其中$z$和$x$具有相同的维度）。
上下文向量$z^{(i)}$是对输入$x^{(1)}$到$x^{(T)}$的加权求和。
上下文向量是特定于某个输入的“上下文”。

不用$x^{(i)}$作为任意输入标记的占位符，让我们考虑第二个输入$x^{(2)}$。
继续用一个具体的例子，我们不考虑占位符$z^{(i)}$，而是考虑第二个输出上下文向量$z^{(2)}$。
第二个上下文向量$z^{(2)}$是对所有输入$x^{(1)}$到$x^{(T)}$根据第二个输入元素$x^{(2)}$进行加权求和。
注意力权重是在计算$z^{(2)}$时决定每个输入元素对加权求和贡献多少的权重。
简而言之，将$z^{(2)}$视为$x^{(2)}$的修改版本，它还包含了与给定任务相关的所有其他输入元素的信息。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- (Please note that the numbers in this figure are truncated to one
digit after the decimal point to reduce visual clutter; similarly, other figures may also contain truncated values)

- By convention, the unnormalized attention weights are referred to as **"attention scores"** whereas the normalized attention scores, which sum to 1, are referred to as **"attention weights"**


- The code below walks through the figure above step by step

<br>

- **Step 1:** compute unnormalized attention scores $\omega$
- Suppose we use the second input token as the query, that is, $q^{(2)} = x^{(2)}$, we compute the unnormalized attention scores via dot products:
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- Above, $\omega$ is the Greek letter "omega" used to symbolize the unnormalized attention scores
    - The subscript "21" in $\omega_{21}$ means that input sequence element 2 was used as a query against input sequence element 1

- Suppose we have the following input sentence that is already embedded in 3-dimensional vectors as described in chapter 3 (we use a very small embedding dimension here for illustration purposes, so that it fits onto the page without line breaks):

In [2]:
import torch

# 定义输入张量，每一行代表一个单词的嵌入向量
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],  # Your     (x^1)
     [0.55, 0.87, 0.66],  # journey  (x^2)
     [0.57, 0.85, 0.64],  # starts   (x^3)
     [0.22, 0.58, 0.33],  # with     (x^4)
     [0.77, 0.25, 0.10],  # one      (x^5)
     [0.05, 0.80, 0.55]]  # step     (x^6)
)

这段文本来自一本关于机器学习和深度学习的书籍，主要内容如下：

1. **数据表示惯例**
   - 在本书中，遵循机器学习和深度学习中的常见惯例，训练样本以行表示，特征值以列表示。对于上面显示的张量，每行代表一个单词，每列代表一个嵌入维度。

2. **本节主要目标**
   - 本节的主要目的是演示如何使用第二个输入序列\(x^{(2)}\)作为查询来计算上下文向量\(z^{(2)}\)。

3. **图中的内容**
   - 图中展示了这个过程的初始步骤，即通过点积运算计算\(x^{(2)}\)与所有其他输入元素之间的注意力分数\(\omega\)。

### 背景知识
在自然语言处理（NLP）中，特别是在Transformer架构等现代模型中，注意力机制是一个关键组件。注意力机制通过计算注意力分数来确定在处理序列数据时应该对输入序列的哪些部分给予更多的关注。

### 总结
这段文本介绍了书中的数据表示惯例，并说明了当前章节的主要目标是演示如何基于特定输入计算上下文向量，且图中展示了计算注意力分数的初始步骤。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- We use input sequence element 2, $x^{(2)}$, as an example to compute context vector $z^{(2)}$; later in this section, we will generalize this to compute all context vectors.
- The first step is to compute the unnormalized attention scores by computing the dot product between the query $x^{(2)}$ and all other input tokens:

In [3]:
# 选择第二个输入向量作为查询向量
query = inputs[1]  # 2nd input token is the query
# 初始化一个空张量，用于存储注意力分数，长度与输入向量的行数相同
attn_scores_2 = torch.empty(inputs.shape[0])
# 循环计算每个输入向量与查询向量的点积，得到注意力分数
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)  # dot product (transpose not necessary here since they are 1-dim vectors)
# 打印注意力分数
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- Side note: a dot product is essentially a shorthand for multiplying two vectors elements-wise and summing the resulting products:

In [4]:
# 初始化结果变量为0
res = 0.
# 使用enumerate函数遍历inputs张量第一行（索引为0）的每个元素
for idx, element in enumerate(inputs[0]):
    # 计算inputs第一行的每个元素与查询向量（query）对应元素的乘积，并累加到res中
    res += inputs[0][idx] * query[idx]
# 打印手动计算的结果
print(res)
# 使用torch.dot函数计算inputs第一行与查询向量（query）的点积，并打印结果
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **步骤2**：对未归一化的注意力分数（“omegas”，$\omega$）进行归一化处理，以使它们的总和为1。
- 以下是一种将未归一化的注意力分数归一化使其总和为1的简单方法（这是一种惯例，对解读结果很有用，且对训练稳定性也很重要）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
# 将未归一化的注意力分数attn_scores_2进行归一化，得到临时的注意力权重attn_weights_2_tmp
# 归一化的方式是用每个未归一化的注意力分数除以所有未归一化注意力分数的总和
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
# 打印输出经过归一化处理后得到的注意力权重
# 这样可以查看在当前查询下，各个输入元素被关注的程度情况
print("Attention weights:", attn_weights_2_tmp)
# 为了验证归一化操作是否正确，计算并打印输出归一化后的注意力权重的总和
# 按照归一化的要求，这个总和应该等于1
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- However, in practice, using the softmax function for normalization, which is better at handling extreme values and has more desirable gradient properties during training, is common and recommended.
- Here's a naive implementation of a softmax function for scaling, which also normalizes the vector elements such that they sum up to 1:

In [6]:
import torch


# 定义一个简单的softmax函数实现
# 该函数接受一个张量x作为输入
def softmax_naive(x):
    """
    这个函数实现了一个简单的softmax操作。
    Softmax函数的作用是将输入张量中的每个元素转换为一个在0到1之间的数，
    并且经过softmax处理后的所有元素之和为1，从而形成一个概率分布。
    这里的实现方式是先对输入张量x的每个元素取指数（torch.exp(x)），
    然后将取指数后的结果除以取指数结果在指定维度（这里是dim=0）上的总和。
    """
    return torch.exp(x) / torch.exp(x).sum(dim=0)
# 使用定义好的softmax_naive函数对之前计算得到的未归一化注意力分数attn_scores_2进行softmax处理
# 得到经过softmax处理后的注意力权重attn_weights_2_naive
attn_weights_2_naive = softmax_naive(attn_scores_2)
# 打印输出经过softmax处理后得到的注意力权重
# 通过查看这些权重值，可以了解在当前查询下各个输入元素被关注的程度情况
print("Attention weights:", attn_weights_2_naive)
# 为了验证经过softmax处理后得到的注意力权重是否满足总和为1的要求
# 计算并打印输出这些注意力权重的总和
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- The naive implementation above can suffer from numerical instability issues for large or small input values due to overflow and underflow issues
- Hence, in practice, it's recommended to use the PyTorch implementation of softmax instead, which has been highly optimized for performance:

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **Step 3**: compute the context vector $z^{(2)}$ by multiplying the embedded input tokens, $x^{(i)}$ with the attention weights and sum the resulting vectors:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
import torch

# 这里将第二个输入标记作为查询来进行后续相关计算
query = inputs[1]  # 2nd input token is the query
# 创建一个与查询向量形状相同的零向量context_vec_2
# 这个向量将用于构建与查询向量相关的上下文向量
context_vec_2 = torch.zeros(query.shape)
# 遍历输入张量inputs中的每一个输入向量x_i及其索引i
for i, x_i in enumerate(inputs):
    # 根据之前计算得到的注意力权重attn_weights_2中对应索引i的权重值
    # 将该权重值与当前输入向量x_i相乘，并累加到上下文向量context_vec_2中
    context_vec_2 += attn_weights_2[i] * x_i
# 打印输出最终构建好的上下文向量context_vec_2
# 该上下文向量综合考虑了各个输入向量按照注意力权重分配后的贡献情况
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Computing attention weights for all input tokens

#### Generalize to all input sequence tokens:

- Above, we computed the attention weights and context vector for input 2 (as illustrated in the highlighted row in the figure below)
- Next, we are generalizing this computation to compute all attention weights and context vectors

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- (Please note that the numbers in this figure are truncated to two
digits after the decimal point to reduce visual clutter; the values in each row should add up to 1.0 or 100%; similarly, digits in other figures are truncated)

- In self-attention, the process starts with the calculation of attention scores, which are subsequently normalized to derive attention weights that total 1
- These attention weights are then utilized to generate the context vectors through a weighted summation of the inputs

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- Apply previous **step 1** to all pairwise elements to compute the unnormalized attention score matrix:

In [9]:
import torch
# 创建一个形状为(6, 6)的空张量attn_scores
# 这个张量将用于存储所有输入向量之间的注意力分数
attn_scores = torch.empty(6, 6)
# 外层循环，遍历输入张量inputs中的每一个输入向量x_i及其索引i
for i, x_i in enumerate(inputs):
    # 内层循环，对于每一个x_i，再次遍历输入张量inputs中的每一个输入向量x_j及其索引j
    for j, x_j in enumerate(inputs):
        # 计算当前的x_i向量与x_j向量的点积
        # 并将结果存储到attn_scores张量的对应位置[i, j]上
        attn_scores[i, j] = torch.dot(x_i, x_j)
# 打印输出计算得到的attn_scores张量
# 该张量展示了所有输入向量两两之间的注意力分数情况
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- We can achieve the same as above more efficiently via matrix multiplication:

In [10]:
import torch
# 使用矩阵乘法运算符 @ 计算输入张量inputs与它自身的转置inputs.T的乘积
# 得到的结果存储在attn_scores张量中，这个结果就是所有输入向量两两之间的注意力分数
attn_scores = inputs @ inputs.T
# 打印输出计算得到的attn_scores张量
# 通过查看该张量，可以了解到所有输入向量两两之间的注意力分数情况
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Similar to **step 2** previously, we normalize each row so that the values in each row sum to 1:

In [11]:
import torch
# 使用PyTorch内置的softmax函数对之前计算得到的注意力分数attn_scores进行softmax操作
# dim=-1表示在最后一个维度上进行softmax运算，目的是将注意力分数在该维度上转换为概率分布
# 即经过softmax处理后，在最后一个维度上的每个元素对应的值会变成一个在0到1之间的数，且该维度上所有元素之和为1
attn_weights = torch.softmax(attn_scores, dim=-1)
# 打印输出经过softmax处理后得到的注意力权重
# 通过查看这些权重值，可以了解在当前设定下各个元素相对应的被关注的程度情况
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- Quick verification that the values in each row indeed sum to 1:

In [12]:

# 将计算结果赋值给row_2_sum变量，用于后续操作或展示
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
# 打印输出带有文字说明的第二行总和的值
print("Row 2 sum:", row_2_sum)
# 针对之前可能已经计算好的attn_weights张量（大概率与注意力权重相关，具体要依据上下文来明确）
# 使用sum函数并指定dim=-1参数，这意味着在该张量的最后一个维度上进行求和操作
# 该操作会将attn_weights张量在每个最后维度（通常可以理解为矩阵的行维度，如果是二维张量的话）上的数据分别求和
# 得到的结果是一个新的张量，其中每个元素代表着attn_weights张量对应行的总和
# 最后打印输出这个包含所有行总和的新张量，以便查看整体的行总和情况
print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- Apply previous **step 3** to compute all context vectors:

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- As a sanity check, the previously computed context vector $z^{(2)} = [0.4419, 0.6515, 0.5683]$ can be found in the 2nd row in above: 

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 Implementing self-attention with trainable weights

- A conceptual framework illustrating how the self-attention mechanism developed in this section integrates into the overall narrative and structure of this book and chapter

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 Computing the attention weights step by step

- In this section, we are implementing the self-attention mechanism that is used in the original transformer architecture, the GPT models, and most other popular LLMs
- This self-attention mechanism is also called "scaled dot-product attention"
- The overall idea is similar to before:
  - We want to compute context vectors as weighted sums over the input vectors specific to a certain input element
  - For the above, we need attention weights
- As you will see, there are only slight differences compared to the basic attention mechanism introduced earlier:
  - The most notable difference is the introduction of weight matrices that are updated during model training
  - These trainable weight matrices are crucial so that the model (specifically, the attention module inside the model) can learn to produce "good" context vectors

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- Implementing the self-attention mechanism step by step, we will start by introducing the three training weight matrices $W_q$, $W_k$, and $W_v$
- These three matrices are used to project the embedded input tokens, $x^{(i)}$, into query, key, and value vectors via matrix multiplication:

  - Query vector: $q^{(i)} = W_q \,x^{(i)}$
  - Key vector: $k^{(i)} = W_k \,x^{(i)}$
  - Value vector: $v^{(i)} = W_v \,x^{(i)}$


- The embedding dimensions of the input $x$ and the query vector $q$ can be the same or different, depending on the model's design and specific implementation
- In GPT models, the input and output dimensions are usually the same, but for illustration purposes, to better follow the computation, we choose different input and output dimensions here:

In [15]:
# 从之前定义好的输入张量inputs中选取第二个输入元素，赋值给x_2
# 这里假设inputs是一个包含多个输入元素的张量，索引从0开始，所以索引为1表示第二个元素
x_2 = inputs[1]  # second input element
# 获取输入张量inputs的第二维的大小，即输入嵌入的尺寸（embedding size）
# 这里假设inputs是一个二维张量，第一维可能表示输入元素的个数，第二维表示每个输入元素的嵌入维度
# 所以通过shape[1]获取到的就是每个输入元素的嵌入维度大小，这里赋值给d_in，并注释说明其含义为输入嵌入尺寸，当前值为3
d_in = inputs.shape[1]  # the input embedding size, d=3
# 定义输出嵌入的尺寸为2，并将其赋值给d_out
# 这个输出嵌入尺寸可能用于后续将输入元素进行某种变换后得到的输出结果的维度设定
# 比如可能是经过与权重矩阵相乘等操作后得到新的嵌入向量，其维度就由这里的d_out指定，这里注释说明其含义为输出嵌入尺寸，当前值为2
d_out = 2  # the output embedding size, d=2

- Below, we initialize the three weight matrices; note that we are setting `requires_grad=False` to reduce clutter in the outputs for illustration purposes, but if we were to use the weight matrices for model training, we would set `requires_grad=True` to update these matrices during model training

In [16]:
import torch
# 设置随机种子为123
# 这样做的目的是为了在后续涉及到随机数生成的操作（如初始化权重矩阵等）时，能够复现相同的随机结果
# 便于调试和对比不同次运行的代码效果是否一致
torch.manual_seed(123)
# 初始化查询权重矩阵W_query
# 使用torch.nn.Parameter将随机生成的张量转换为可学习的参数（在实际模型训练中，若设置requires_grad=True则可参与梯度更新）
# torch.rand(d_in, d_out)会生成一个形状为(d_in, d_out)的随机张量，这里d_in是输入嵌入尺寸（之前获取到的值），d_out是输出嵌入尺寸（之前定义的值）
# 由于当前是为了示例展示目的，将requires_grad设置为False，即该权重矩阵在当前示例中不参与梯度更新，只是用于展示相关计算过程
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
# 初始化键权重矩阵W_key
# 同样的方式，使用torch.nn.Parameter将随机生成的张量转换为可学习的参数，形状也是根据输入和输出嵌入尺寸确定
# 并且也将requires_grad设置为False，用于示例展示相关计算过程，不参与梯度更新
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
# 初始化值权重矩阵W_value
# 按照与上述两个矩阵相同的方式进行初始化，生成形状为(d_in, d_out)的随机张量并转换为可学习的参数
# 同样设置requires_grad为False，以便在示例展示时简化输出，避免因梯度计算带来的繁杂信息
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- Next we compute the query, key, and value vectors:

In [17]:
# 计算与第二个输入元素（x_2）相关的查询向量（query_2）
# 通过将第二个输入元素x_2与查询权重矩阵W_query进行矩阵乘法运算得到查询向量
# 这里在变量名后添加_2是为了表明该查询向量是针对第二个输入元素进行计算得到的
query_2 = x_2 @ W_query  # _2 because it's with respect to the 2nd input element
# 计算与第二个输入元素（x_2）相关的键向量（key_2）
# 同样是将第二个输入元素x_2与键权重矩阵W_key进行矩阵乘法运算得到键向量
key_2 = x_2 @ W_key
# 计算与第二个输入元素（x_2）相关的值向量（value_2）
# 把第二个输入元素x_2与值权重矩阵W_value进行矩阵乘法运算得到值向量
value_2 = x_2 @ W_value
# 打印输出计算得到的查询向量query_2
# 通过查看打印结果，可以了解到针对第二个输入元素计算出的查询向量的具体数值情况
print(query_2)

tensor([0.4306, 1.4551])


- As we can see below, we successfully projected the 6 input tokens from a 3D onto a 2D embedding space:

In [18]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- In the next step, **step 2**, we compute the unnormalized attention scores by computing the dot product between the query and each key vector:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [19]:
keys_2 = keys[1]  # Python starts index at 0
# 计算查询向量query_2和选取的第二个键向量keys_2之间的点积
# 得到的结果attn_score_22就是针对第二个输入元素（之前计算query_2时所基于的那个输入元素）
attn_score_22 = query_2.dot(keys_2)
# 打印输出计算得到的注意力分数attn_score_22
# 通过查看这个值，可以了解到当前查询向量与所选取的键向量之间的相关性程度
print(attn_score_22)

tensor(1.8524)


- Since we have 6 inputs, we have 6 attention scores for the given query vector:

In [20]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- Next, in **step 3**, we compute the attention weights (normalized attention scores that sum up to 1) using the softmax function we used earlier
- The difference to earlier is that we now scale the attention scores by dividing them by the square root of the embedding dimension, $\sqrt{d_k}$ (i.e., `d_k**0.5`):

In [21]:
# 获取键向量'keys'的第二维的大小，也就是每个键向量的维度数量，并将其赋值给变量'd_k'。
d_k = keys.shape[1]
# 计算注意力权重'attn_weights_2'。
# 首先，将之前已经计算好的注意力分数'attn_scores_2'除以'd_k'的0.5次方（即对'd_k'进行开根号操作）。
# 这样做的目的是为了对注意力分数进行一种标准化或者缩放处理。在自注意力机制等相关场景下，这种缩放操作有助于提高计算的数值稳定性，避免出现数值过大或过小导致的一些问题，比如梯度消失或梯度爆炸等情况，同时也能在一定程度上优化模型的训练效果。
# 然后，使用PyTorch库中的'softmax'函数对经过上述缩放处理后的结果，在最后一个维度（通过'dim=-1'指定）上进行'softmax'操作。
#'softmax'函数的作用是将输入的数值转换为一个概率分布。具体来说，经过'softmax'处理后，在指定维度（这里是最后一个维度）上的每个元素对应的值会变成一个在0到1之间的数，并且该维度上所有元素之和为1。
# 这样得到的'attn_weights_2'就表示了各个元素被关注的程度，是以概率分布的形式呈现出来的，每个概率值对应着相应元素在当前情境下被关注的可能性大小。
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
# 打印输出经过上述一系列计算得到的注意力权重'attn_weights_2'。
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- In **step 4**, we now compute the context vector for input query vector 2:

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 Implementing a compact SelfAttention class

- Putting it all together, we can implement the self-attention mechanism as follows:

In [23]:
import torch
import torch.nn as nn
# 定义一个名为SelfAttention_v1的类，它继承自nn.Module，这是PyTorch中用于定义神经网络模块的基类
# 这个类实现了一个简单版本的自注意力机制
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        """
        类的初始化方法，用于创建自注意力机制所需的参数。
        Args:
            d_in (int): 输入数据的嵌入维度大小。
            d_out (int): 输出数据的嵌入维度大小，也就是经过自注意力机制处理后希望得到的输出维度。
        """
        super().__init__()
        # 初始化查询权重矩阵W_query，它是一个可学习的参数
        # 使用torch.rand(d_in, d_out)随机生成一个形状为(d_in, d_out)的张量，然后通过nn.Parameter将其转换为可学习的参数
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        # 初始化键权重矩阵W_key，同样是一个可学习的参数，生成方式与W_query类似
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        # 初始化值权重矩阵W_value，也是可学习的参数，用于后续计算值向量
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    def forward(self, x):
        """
        前向传播方法，定义了数据在自注意力机制中的流动和计算过程。
        Args:
            x (torch.Tensor): 输入张量，其形状通常为(batch_size, sequence_length, d_in)，其中batch_size表示批次大小，sequence_length表示序列长度，d_in是输入嵌入维度。
        Returns:
            torch.Tensor: 经过自注意力机制处理后得到的上下文向量，其形状通常为(batch_size, sequence_length, d_out)。
        """
        # 通过输入张量x与键权重矩阵W_key进行矩阵乘法运算，得到键向量keys
        # 这里的计算会对输入张量中的每个元素（假设输入张量形状为(batch_size, sequence_length, d_in)）分别进行操作，得到对应的键向量
        keys = x @ self.W_key
        # 同样地，通过输入张量x与查询权重矩阵W_query进行矩阵乘法运算，得到查询向量queries
        queries = x @ self.W_query
        # 通过输入张量x与值权重矩阵W_value进行矩阵乘法运算，得到值向量values
        values = x @ self.W_value
        # 计算注意力分数attn_scores
        # 通过查询向量queries与键向量keys的转置keys.T进行矩阵乘法运算得到
        # 这个结果可以理解为衡量了不同输入元素之间的相关性程度，也就是每个查询向量与每个键向量之间的一种关联度量，这里记为omega
        attn_scores = queries @ keys.T  # omega
        # 计算注意力权重attn_weights
        # 首先将注意力分数attn_scores除以键向量keys的最后一维（通常是键向量的维度，假设键向量形状为(batch_size, sequence_length, d_key)，这里的d_key就是最后一维的大小）的0.5次方（即开根号）
        # 这样做是为了对注意力分数进行一定的缩放处理，有助于提高数值稳定性和训练效果等
        # 然后使用torch.softmax函数对缩放后的结果在最后一个维度（dim=-1）上进行softmax操作
        # softmax操作会将输入的数值转换为一个概率分布，使得在最后一个维度上的每个元素对应的值会变成一个在0到1之间的数，且该维度上所有元素之和为1
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        # 通过注意力权重attn_weights与值向量values进行矩阵乘法运算，得到上下文向量context_vec
        # 这个操作会根据注意力权重对值向量进行加权求和，从而得到综合考虑了不同输入元素被关注程度的上下文向量
        context_vec = attn_weights @ values
        return context_vec
# 设置随机种子为123，这样做的目的是为了在后续涉及到随机数生成的操作（如初始化权重矩阵等）时，能够复现相同的随机结果
# 便于调试和对比不同次运行的代码效果是否一致
torch.manual_seed(123)
# 创建一个SelfAttention_v1类的实例sa_v1，传入输入嵌入维度d_in和输出嵌入维度d_out作为参数
# 此时会自动调用类的__init__方法进行初始化操作，创建所需的权重矩阵等参数
sa_v1 = SelfAttention_v1(d_in, d_out)
# 调用sa_v1实例的__call__方法（在Python中，对象后面加括号会调用其__call__方法，对于继承自nn.Module的类，__call__方法会自动调用forward方法）
# 将inputs作为参数传入，这里假设inputs是一个符合自注意力机制输入要求的张量，比如形状可能为(batch_size, sequence_length, d_in)
# 然后打印输出经过自注意力机制处理后得到的结果，也就是由sa_v1的forward方法返回的上下文向量
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- We can streamline the implementation above using PyTorch's Linear layers, which are equivalent to a matrix multiplication if we disable the bias units
- Another big advantage of using `nn.Linear` over our manual `nn.Parameter(torch.rand(...)` approach is that `nn.Linear` has a preferred weight initialization scheme, which leads to more stable model training

In [24]:
import torch
import torch.nn as nn

# 定义一个名为SelfAttention_v2的类，它继承自nn.Module，这是PyTorch中用于定义神经网络模块的基类
# 这个类实现了另一种版本的自注意力机制
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        """
        类的初始化方法，用于创建自注意力机制所需的参数。
        Args:
            d_in (int): 输入数据的嵌入维度大小。
            d_out (int): 输出数据的嵌入维度大小，也就是经过自注意力机制处理后希望得到的输出维度。
            qkv_bias (bool): 是否给查询（query）、键（key）、值（value）对应的线性变换层添加偏置项，默认为False。
        """
        super().__init__()
        # 使用nn.Linear创建查询权重矩阵对应的线性变换层W_query
        # nn.Linear会根据输入维度d_in和输出维度d_out创建一个全连接层，用于将输入数据进行线性变换
        # 如果qkv_bias为True，则会给这个线性变换层添加偏置项，否则不添加
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 同样地，使用nn.Linear创建键权重矩阵对应的线性变换层W_key
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 创建值权重矩阵对应的线性变换层W_value
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    def forward(self, x):
        """
        前向传播方法，定义了数据在自注意力机制中的流动和计算过程。
        Args:
            x (torch.Tensor): 输入张量，其形状通常为(batch_size, sequence_length, d_in)，其中batch_size表示批次大小，sequence_length表示序列长度，d_in是输入嵌入维度。
        Returns:
            torch.Tensor): 经过自注意力机制处理后得到的上下文向量，其形状通常为(batch_size, sequence_length, d_out)。
        """
        # 通过键权重矩阵对应的线性变换层W_key对输入张量x进行线性变换，得到键向量keys
        keys = self.W_key(x)
        # 通过查询权重矩阵对应的线性变换层W_query对输入张量x进行线性变换，得到查询向量queries
        queries = self.W_query(x)
        # 通过值权重矩阵对应的线性变换层W_value对输入张量x进行线性变换，得到值向量values
        values = self.W_value(x)
        # 计算注意力分数attn_scores
        # 通过查询向量queries与键向量keys的转置keys.T进行矩阵乘法运算得到
        # 这个结果可以理解为衡量了不同输入元素之间的相关性程度，也就是每个查询向量与每个键向量之间的一种关联度量
        attn_scores = queries @ keys.T
        # 计算注意力权重attn_weights
        # 首先将注意力分数attn_scores除以键向量keys的最后一维（通常是键向量的维度，假设键向量形状为(batch_size, sequence_length, d_key)，这里的d_key就是最后一维的大小）的0.5次方（即开根号）
        # 这样做是为了对注意力分数进行一定的缩放处理，有助于提高数值稳定性和训练效果等
        # 然后使用torch.softmax函数对缩放后的结果在最后一个维度（dim=-1）上进行softmax操作
        # softmax操作会将输入的数值转换为一个概率分布，使得在最后一个维度上的每个元素对应的值会变成一个在0到1之间的数，且该维度上所有元素之和为1
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # 通过注意力权重attn_weights与值向量values进行矩阵乘法运算，得到上下文向量context_vec
        # 这个操作会根据注意力权重对值向量进行加权求和，从而得到综合考虑了不同输入元素被关注程度的上下文向量
        context_vec = attn_weights @ values
        return context_vec
torch.manual_seed(789)
# 创建一个SelfAttention_v2类的实例sa_v2，传入输入嵌入维度d_in和输出嵌入维度d_out作为参数
# 此时会自动调用类的__init__方法进行初始化操作，创建所需的权重矩阵等参数
sa_v2 = SelfAttention_v2(d_in, d_out)
# 调用sa_v2实例的__call__方法（在Python中，对象后面加括号会调用其__call__方法，对于继承自nn.Module的类，__call__方法会自动调用forward方法）
# 将inputs作为参数传入，这里假设inputs是一个符合自注意力机制输入要求的张量，比如形状可能为(batch_size, sequence_length, d_in)
# 然后打印输出经过自注意力机制处理后得到的结果，也就是由sa_v2的forward方法返回的上下文向量1
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- Note that `SelfAttention_v1` and `SelfAttention_v2` give different outputs because they use different initial weights for the weight matrices

## 3.5 Hiding future words with causal attention

- In causal attention, the attention weights above the diagonal are masked, ensuring that for any given input, the LLM is unable to utilize future tokens while calculating the context vectors with the attention weight

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 Applying a causal attention mask

- In this section, we are converting the previous self-attention mechanism into a causal self-attention mechanism
- Causal self-attention ensures that the model's prediction for a certain position in a sequence is only dependent on the known outputs at previous positions, not on future positions
- In simpler words, this ensures that each next word prediction should only depend on the preceding words
- To achieve this, for each given token, we mask out the future tokens (the ones that come after the current token in the input text):

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- To illustrate and implement causal self-attention, let's work with the attention scores and weights from the previous section: 

In [25]:
# 为了方便，重用上一部分中SelfAttention_v2对象的查询（query）和键（key）权重矩阵
# 这里假设sa_v2是已经定义好的实现了SelfAttention_v2机制的对象，通过它的W_query方法，传入输入数据inputs，计算得到查询向量queries
queries = sa_v2.W_query(inputs)  
# 同样通过sa_v2的W_key方法，传入输入数据inputs，计算得到键向量keys
keys = sa_v2.W_key(inputs)  
# 计算注意力分数（attention scores）
# 通过查询向量queries和键向量keys的转置（keys.T）进行矩阵乘法（@操作符在PyTorch中用于矩阵乘法）
# 得到的attn_scores矩阵中的每个元素表示了查询向量中的每个元素与键向量中的每个元素之间的匹配程度
attn_scores = queries @ keys.T  
# 计算注意力权重（attention weights）
# 首先对注意力分数attn_scores进行缩放操作，除以keys.shape[-1]的平方根
# 这里的keys.shape[-1]通常表示键向量的最后一个维度的大小（在常见的注意力机制实现中，可能是键向量的特征维度大小）
# 除以它的平方根是为了在进行后续的softmax操作时，使得注意力分数的分布更加合理，避免数值过大或过小导致softmax结果出现极端情况（例如梯度消失或爆炸等问题）
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)  
# 打印出计算得到的注意力权重，以便查看其具体数值
print(attn_weights)  

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- The simplest way to mask out future attention weights is by creating a mask via PyTorch's tril function with elements below the main diagonal (including the diagonal itself) set to 1 and above the main diagonal set to 0:

In [26]:
import torch
# 获取注意力分数张量的第一个维度大小，通常用于表示序列长度等相关概念，这里将其赋值给context_length
context_length = attn_scores.shape[0]
# 创建一个形状为(context_length, context_length)的全1矩阵，例如若context_length为5，就会创建一个5x5的全1矩阵
# 这个全1矩阵后续会被处理成下三角矩阵形式来作为掩码使用
mask_simple = torch.tril(torch.ones(context_length, context_length))
# 打印出创建好的下三角矩阵掩码，以便查看其具体内容，在实际运行环境中会按照张量格式输出具体数值情况
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- Then, we can multiply the attention weights with this mask to zero out the attention scores above the diagonal:

In [27]:
# 获取注意力分数attn_scores矩阵在第一维度（通常在相关应用场景下，这个维度可能对应着序列的长度、时间步长等类似概念）上的大小
# 将其赋值给context_length变量，以便后续基于这个长度值来进行一些相关的矩阵操作等
context_length = attn_scores.shape[0]  
# 例如，如果attn_scores是一个形状为 (序列长度, 其他维度1, 其他维度2,...) 的矩阵，这里就是获取了序列长度的值
# 使用torch.tril函数来创建一个特定形状的矩阵，这里是基于前面获取到的context_length作为边长创建一个方阵
# 首先通过torch.ones函数创建一个所有元素都为1的矩阵，其形状为 (context_length, context_length)
# 然后将这个全1矩阵传入torch.tril函数中，torch.tril函数会将输入矩阵的上三角部分（不包括主对角线）的元素都设置为0
# 只保留其下三角部分（包括主对角线）的元素，最终得到的mask_simple矩阵就是一个下三角矩阵，主对角线及其下方元素为1，上方元素为0
mask_simple = torch.tril(torch.ones(context_length, context_length))  
# 打印出创建好的这个下三角矩阵mask_simple，这样可以直观地查看其具体的元素分布情况
# 比如在一些基于注意力机制处理序列数据的场景中，这个下三角矩阵可能会用于后续对注意力分数进行掩码操作（限制某些位置的注意力计算等），通过打印可以确认其是否符合预期的形式
print(mask_simple)  

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- However, if the mask were applied after softmax, like above, it would disrupt the probability distribution created by softmax
- Softmax ensures that all output values sum to 1
- Masking after softmax would require re-normalizing the outputs to sum to 1 again, which complicates the process and might lead to unintended effects

- To make sure that the rows sum to 1, we can normalize the attention weights as follows:

In [28]:
# 计算 masked_simple 在最后一个维度（dim=-1）上的元素和，并保持维度不变（keepdim=True）
# 例如，如果 masked_simple 是一个形状为 (batch_size, num_heads, sequence_length, sequence_length) 的张量
# 那么 sum(dim=-1) 就是对最后一个 sequence_length 维度上的元素进行求和操作
# 计算结果的形状会变为 (batch_size, num_heads, sequence_length, 1)，因为 keepdim=True 保持了求和后的维度
row_sums = masked_simple.sum(dim=-1, keepdim=True)
# 将 masked_simple 张量的每个元素除以对应的行和（row_sums），实现对 masked_simple 的归一化操作
# 这样做的目的通常是为了让每行的元素经过某种变换后总和为1，符合一些概率分布相关的计算要求或者其他归一化需求
# 例如，在注意力机制中，经过这样的归一化后，每行的元素可以被看作是对应位置的相对权重等情况
masked_simple_norm = masked_simple / row_sums
# 打印出经过归一化处理后的张量 masked_simple_norm，以便查看具体的归一化结果
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- While we are technically done with coding the causal attention mechanism now, let's briefly look at a more efficient approach to achieve the same as above
- So, instead of zeroing out attention weights above the diagonal and renormalizing the results, we can mask the unnormalized attention scores above the diagonal with negative infinity before they enter the softmax function:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
import torch
# 创建一个上三角掩码矩阵，用于后续对注意力分数进行处理
# 首先创建一个大小为context_length行context_length列的全1矩阵
# 这里的context_length通常是与注意力计算相关的序列长度等维度信息
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# 解释如下：
# 例如，如果context_length为5，那么torch.ones(context_length, context_length)创建的矩阵就是：
# [[1., 1., 1., 1., 1.],
#  [1., 1., 1., 1., 1.],
#  [1., 1., 1., 1., 1.],
#  [1., 1., 1., 1., 1.],
#  [1., 1., 1., 1., 1.]]
# 接着，torch.triu函数用于获取这个全1矩阵的上三角部分（不包括主对角线），diagonal=1表示从主对角线往上一格开始取上三角部分
# 对于上述假设的5x5矩阵，经过torch.triu处理后得到的矩阵mask为：
# [[0., 1., 1., 1., 1.],
#  [0., 0., 1., 1., 1.],
#  [0., 0., 0., 1., 1.],
#  [0., 0., 0., 0., 1.],
#  [0., 0., 0., 0., 0.]]
# 使用创建的上三角掩码对注意力分数attn_scores进行填充操作
# 首先将掩码矩阵mask转换为布尔类型，以便于在masked_fill函数中使用
# 在布尔类型下，矩阵中非零元素对应的位置为True，零元素对应的位置为False
# 然后，根据布尔类型的掩码矩阵，将attn_scores中对应True位置的元素填充为 -torch.inf（负无穷）
# 这样就可以把attn_scores中对应于上三角部分（不包括主对角线）的注意力分数都设置为负无穷
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
# 打印出经过掩码处理后得到的新的张量masked，以便查看处理后的注意力分数情况
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- As we can see below, now the attention weights in each row correctly sum to 1 again:

In [30]:
# 首先，对masked进行一个缩放操作，除以keys.shape[-1]的平方根（这里的keys应该是在注意力机制相关计算中涉及到的另一个张量，可能是键向量等相关概念）
# 这个缩放操作是注意力机制中的常见操作，目的是为了调整数值范围，使得后续的softmax计算更加合理和稳定
# keys.shape[-1]表示获取keys张量的最后一个维度的大小，然后对其开平方根并用于除法运算
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
# 例如，如果keys是一个形状为(batch_size, num_heads, sequence_length, key_dim)的张量，那么keys.shape[-1]就是key_dim
# 假设key_dim的值为64，那么keys.shape[-1]**0.5就是8，就会将masked中的每个元素除以8后再进行softmax操作
# 接着，使用torch.softmax函数对经过缩放处理后的masked进行softmax运算
# softmax函数会将输入的张量在指定的维度（这里是dim=-1，即最后一个维度）上进行归一化处理
# 使得该维度上的每个元素都被转换为一个介于0和1之间的概率值，并且该维度上所有元素的和为1
# 经过softmax运算后得到的结果就是注意力权重attn_weights，它表示在注意力机制中每个位置相对于其他位置的重要性程度，类似于一种概率分布
# 打印出计算得到的注意力权重attn_weights，以便查看具体的数值情况和概率分布情况
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 Masking additional attention weights with dropout

- In addition, we also apply dropout to reduce overfitting during training
- Dropout can be applied in several places:
  - for example, after computing the attention weights;
  - or after multiplying the attention weights with the value vectors
- Here, we will apply the dropout mask after computing the attention weights because it's more common

- Furthermore, in this specific example, we use a dropout rate of 50%, which means randomly masking out half of the attention weights. (When we train the GPT model later, we will use a lower dropout rate, such as 0.1 or 0.2

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- If we apply a dropout rate of 0.5 (50%), the non-dropped values will be scaled accordingly by a factor of 1/0.5 = 2.

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
import torch
import torch.nn.functional as F  # 假设这里的dropout函数是从torch.nn.functional中引入的，实际情况可能需要根据具体代码环境确定
# 设置随机种子为123
torch.manual_seed(123)
# 假设attn_weights是之前经过一系列注意力机制相关计算得到的注意力权重张量，其形状可能类似于 (batch_size, num_heads, sequence_length, sequence_length)
# 这里将对attn_weights进行dropout操作。
# Dropout是一种广泛应用于深度学习中的正则化技术，其主要目的是防止模型过拟合。
# 它通过在训练过程中随机地将输入张量中的一些元素置为0，来模拟神经元之间的随机连接丢失情况，从而减少神经元之间的过度依赖，增强模型的泛化能力。
# 这里调用F.dropout函数对attn_weights进行dropout处理，然后将处理后的结果打印出来以便查看。
# 在实际应用中，F.dropout函数通常会接受一些参数，比如p表示dropout的概率（即每个元素被置为0的概率），一般默认为0.5，但也可以根据具体需求进行设置。
# 另外，还有一些参数如inplace（是否在原张量上进行操作）等，不过这里没有明确给出这些参数的设置情况，我们假设是按照默认设置进行操作的。
print(F.dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- Note that the resulting dropout outputs may look different depending on your operating system; you can read more about this inconsistency [here on the PyTorch issue tracker](https://github.com/pytorch/pytorch/issues/121595)

### 3.5.3 Implementing a compact causal self-attention class

- Now, we are ready to implement a working implementation of self-attention, including the causal and dropout masks
- One more thing is to implement the code to handle batches consisting of more than one input so that our `CausalAttention` class supports the batch outputs produced by the data loader we implemented in chapter 2
- For simplicity, to simulate such batch input, we duplicate the input text example:

In [33]:
# 这里将同一个inputs张量重复两次，并沿着新的维度dim=0进行堆叠操作。
# torch.stack函数用于沿着一个新的维度拼接多个张量。在这里，它将两个inputs张量拼接在一起，新的维度为dim=0。
batch = torch.stack((inputs, inputs), dim=0)
# 然后打印出堆叠后得到的batch张量的形状。根据注释可知，每个inputs有6个token（可以理解为6个元素或者序列中的6个单元等，具体含义取决于应用场景），
# 并且每个token的嵌入维度是3。当进行torch.stack((inputs, inputs), dim=0)操作后，
# 得到的batch张量在dim=0这个维度上会有2个元素（因为堆叠了两个inputs），在原来inputs的token维度（这里假设为dim=1）上依然是6个token，
# 在每个token的嵌入维度（这里假设为dim=2）上依然是3。
# 所以打印出batch.shape的结果应该是 (2, 6, 3)，表示这个batch张量有2个样本（对应堆叠的两个inputs），每个样本有6个token，每个token的嵌入维度是3。
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [34]:
import torch
import torch.nn as nn
# 定义CausalAttention类，继承自nn.Module，用于实现因果注意力机制
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        """
        构造函数，用于初始化CausalAttention类的实例
        :param d_in: 输入维度
        :param d_out: 输出维度
        :param context_length: 上下文长度，用于确定掩码矩阵的大小
        :param dropout: dropout概率，用于防止过拟合
        :param qkv_bias: 是否在生成查询、键、值向量的线性层中添加偏置项，默认为False
        """
        super().__init__()
        # 保存输出维度，用于后续计算和操作
        self.d_out = d_out
        # 定义用于生成查询向量的线性层
        # 将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 定义用于生成键向量的线性层
        # 同样将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 定义用于生成值向量的线性层
        # 也是将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 创建一个Dropout层实例，用于在训练过程中对注意力权重进行随机丢弃操作
        # dropout参数指定了每个元素被丢弃的概率，以此来防止过拟合
        self.dropout = nn.Dropout(dropout)
        # 创建一个上三角掩码矩阵，并将其注册为模块的缓冲区（buffer）
        # 这个掩码矩阵用于在注意力计算过程中屏蔽掉未来位置的信息，实现因果关系的约束
        # 先创建一个大小为context_length行context_length列的全1矩阵，然后通过torch.triu函数取其指定的上三角部分（不包括主对角线）
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
    def forward(self, x):
        """
        前向传播函数，用于定义数据在模型中的流动和计算过程

        :param x: 输入张量，形状为(b, num_tokens, d_in)，其中b是批处理大小，num_tokens是每个样本中的标记数量，d_in是输入维度
        :return: 经过注意力加权后的上下文向量
        """
        b, num_tokens, d_in = x.shape  # 获取输入张量的形状信息，包括批处理大小、标记数量和输入维度
        # 通过对应的线性层对输入x进行线性变换，生成键向量
        keys = self.W_key(x)
        # 通过对应的线性层对输入x进行线性变换，生成查询向量
        queries = self.W_query(x)
        # 通过对应的线性层对输入x进行线性变换，生成值向量
        values = self.W_value(x)
        # 计算注意力分数，通过查询向量与键向量的转置进行矩阵乘法运算
        attn_scores = queries @ keys.transpose(1, 2)
        # 使用掩码矩阵对注意力分数进行填充操作，将对应于未来位置信息的分数设置为负无穷
        # 这里通过切片操作`:num_tokens`来确保在批处理中标记数量小于支持的上下文大小时，也能正确应用掩码
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        # 对经过掩码处理后的注意力分数进行softmax操作，得到注意力权重
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        # 对注意力权重进行dropout操作，随机丢弃一些元素，防止过拟合
        attn_weights = self.dropout(attn_weights)
        # 通过注意力权重与值向量进行矩阵乘法运算，得到经过注意力加权后的上下文向量
        context_vec = attn_weights @ values
        return context_vec
torch.manual_seed(123)
# 获取batch张量的第二个维度大小作为上下文长度，这里假设batch是已经定义好的输入数据张量
context_length = batch.shape[1]
# 创建CausalAttention类的实例，传入输入维度、输出维度、上下文长度和dropout概率等参数
ca = CausalAttention(d_in, d_out, context_length, 0.0

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Note that dropout is only applied during training, not during inference

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 Extending single-head attention to multi-head attention

### 3.6.1 Stacking multiple single-head attention layers

- Below is a summary of the self-attention implemented previously (causal and dropout masks not shown for simplicity)

- This is also called single-head attention:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- We simply stack multiple single-head attention modules to obtain a multi-head attention module:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- The main idea behind multi-head attention is to run the attention mechanism multiple times (in parallel) with different, learned linear projections. This allows the model to jointly attend to information from different representation subspaces at different positions.

In [35]:
import torch
import torch.nn as nn
# 定义MultiHeadAttentionWrapper类，继承自nn.Module，用于实现多头注意力机制的包装
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        构造函数，用于初始化MultiHeadAttentionWrapper类的实例
        :param d_in: 输入维度
        :param d_out: 输出维度
        :param context_length: 上下文长度，用于确定每个头的因果注意力机制中的掩码矩阵等相关参数
        :param dropout: dropout概率，用于防止过拟合
        :param num_heads: 头的数量，即同时运行的因果注意力机制的数量
        :param qkv_bias: 是否在每个头生成查询、键、值向量的线性层中添加偏置项，默认为False
        """
        super().__init__()
        # 创建一个ModuleList，其中包含多个CausalAttention实例，每个实例代表一个头的因果注意力机制
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )
    def forward(self, x):
        """
        前向传播函数，用于定义数据在模型中的流动和计算过程
        :param x: 输入张量，形状为(b, num_tokens, d_in)，其中b是批处理大小，num_tokens是每个样本中的标记数量，d_in是输入维度
        :return: 将每个头的输出沿着最后一个维度拼接起来得到的结果张量
        """
        # 对每个头的因果注意力机制分别进行前向传播，并将结果沿着最后一个维度（dim=-1）进行拼接
        return torch.cat([head(x) for head in self.heads], dim=-1)
# 设置随机种子为123，确保在涉及随机操作（如后续可能的dropout操作）时能得到可重复的结果
torch.manual_seed(123)
# 获取batch张量的第二个维度大小作为上下文长度，这里假设batch是已经定义好的输入数据张量，该维度表示标记的数量
context_length = batch.shape[1] 
# 定义输入维度和输出维度
d_in, d_out = 3, 2
# 创建MultiHeadAttentionWrapper类的实例，传入输入维度、输出维度、上下文长度、dropout概率和头的数量等参数
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)
# 将batch张量传入MultiHeadAttentionWrapper实例的前向传播函数中，得到经过多头注意力机制处理后的上下文向量
context_vecs = mha(batch)
# 打印出经过多头注意力机制处理后的上下文向量
print(context_vecs)
# 打印出经过多头注意力机制处理后的上下文向量的形状
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- In the implementation above, the embedding dimension is 4, because we `d_out=2` as the embedding dimension for the key, query, and value vectors as well as the context vector. And since we have 2 attention heads, we have the output embedding dimension 2*2=4

### 3.6.2 Implementing multi-head attention with weight splits

- While the above is an intuitive and fully functional implementation of multi-head attention (wrapping the single-head attention `CausalAttention` implementation from earlier), we can write a stand-alone class called `MultiHeadAttention` to achieve the same

- We don't concatenate single attention heads for this stand-alone `MultiHeadAttention` class
- Instead, we create single W_query, W_key, and W_value weight matrices and then split those into individual matrices for each attention head:

In [36]:
import torch
import torch.nn as nn
# 定义MultiHeadAttention类，继承自nn.Module，用于实现多头注意力机制
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        构造函数，用于初始化MultiHeadAttention类的实例
        :param d_in: 输入维度
        :param d_out: 输出维度
        :param context_length: 上下文长度，用于确定掩码矩阵大小等相关参数
        :param dropout: dropout概率，用于防止过拟合
        :param num_heads: 头的数量，即同时运行的注意力机制的数量
        :param qkv_bias: 是否在生成查询、键、值向量的线性层中添加偏置项，默认为False
        """
        super().__init__()
        # 检查输出维度d_out是否能被头的数量num_heads整除，若不能则抛出异常
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        # 保存输出维度
        self.d_out = d_out
        # 保存头的数量
        self.num_heads = num_heads
        # 计算每个头的维度，通过将输出维度除以头的数量得到
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim
        # 定义用于生成查询向量的线性层，将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 定义用于生成键向量的线性层，同样将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 定义用于生成值向量的线性层，也是将输入维度d_in转换为输出维度d_out，可根据qkv_bias决定是否添加偏置项
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 定义一个线性层，用于将各个头的输出进行组合，输入维度为d_out，输出维度也为d_out
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        # 创建一个Dropout层实例，用于在训练过程中对注意力权重进行随机丢弃操作，以防止过拟合
        self.dropout = nn.Dropout(d.dropout)
        # 创建一个上三角掩码矩阵，并将其注册为模块的缓冲区（buffer）
        # 这个掩码矩阵用于在注意力计算过程中屏蔽掉未来位置的信息，实现因果关系的约束
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
    def forward(self, x):
        """
        前向传播函数，用于定义数据在模型中的流动和计算过程
        :param x: 输入张量，形状为(batch_size, num_tokens, d_in)，其中batch_size是批处理大小，num_tokens是每个样本中的标记数量，d_in是输入维度
        :return: 经过多头注意力机制处理后的上下文向量
        """
        b, num_tokens, d_in = x.shape
        # 通过对应的线性层对输入x进行线性变换，生成键向量，形状为(batch_size, num_tokens, d_out)
        keys = self.W_key(x)
        # 通过对应的线性层对输入x进行线性变换，生成查询向量
        queries = self.W_query(x)
        # 通过对应的线性层对输入x进行线性变换，生成值向量
        values = self.W_value(x)
        # 我们通过添加一个`num_heads`维度来隐式地拆分矩阵
        # 将最后一个维度展开：(batch_size, num_tokens, d_out) -> (batch_size, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        # 对上述拆分后的张量进行转置操作：(batch_size, num_tokens, num_heads, head_dim) -> (batch_size, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        # 计算带有因果掩码的缩放点积注意力（即自注意力）
        # 对每个头进行点积操作，计算注意力分数
        attn_scores = queries @ keys.transpose(2, 3)
        # 将原始掩码截断为与当前输入的标记数量相同，并转换为布尔类型
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # 使用掩码来填充注意力分数，将对应于未来位置信息的分数设置为负无穷
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        # 对经过掩码处理后的注意力分数进行softmax操作，得到注意力权重
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        # 形状为(batch_size, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)
        # 组合各个头的输出，其中self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # 可选的投影操作
        return context_vec
# 设置随机种子为123，确保在涉及随机操作（如后续的dropout操作）时能得到可重复的结果
torch.manual_seed(123)
# 获取batch张量的形状信息，分别赋值给batch_size、context_length和d_in
batch_size, context_length, d_in = batch.shape
# 定义输出维度
d_out = 2
# 创建MultiHeadAttention类的实例，传入输入维度、输出维度、上下文长度、dropout概率和头的数量等参数
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
# 将batch张量传入MultiHeadAttention实例的前向传播函数中，得到经过多头注意力机制处理后的上下文向量
context_vecs = mha(batch)
# 打印出经过多头注意力机制处理后的上下文向量
print(context_vecs)
# 打印出经过多头注意力机制处理后的上下文向量的形状
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Note that the above is essentially a rewritten version of `MultiHeadAttentionWrapper` that is more efficient
- The resulting output looks a bit different since the random weight initializations differ, but both are fully functional implementations that can be used in the GPT class we will implement in the upcoming chapters
- Note that in addition, we added a linear projection layer (`self.out_proj `) to the `MultiHeadAttention` class above. This is simply a linear transformation that doesn't change the dimensions. It's a standard convention to use such a projection layer in LLM implementation, but it's not strictly necessary (recent research has shown that it can be removed without affecting the modeling performance; see the further reading section at the end of this chapter)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- Note that if you are interested in a compact and efficient implementation of the above, you can also consider the [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) class in PyTorch

- Since the above implementation may look a bit complex at first glance, let's look at what happens when executing `attn_scores = queries @ keys.transpose(2, 3)`:

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- In this case, the matrix multiplication implementation in PyTorch will handle the 4-dimensional input tensor so that the matrix multiplication is carried out between the 2 last dimensions (num_tokens, head_dim) and then repeated for the individual heads 

- For instance, the following becomes a more compact way to compute the matrix multiplication for each head separately:

In [38]:
# 假设已经定义好了形状为(batch_size, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)的张量a
# 这里分别提取出张量a中的第一个头（索引为0）和第一个样本（索引为0，因为batch_size为1）的数据部分，即第一个头对应的矩阵
first_head = a[0, 0, :, :]
# 此时first_head的形状为(num_tokens, head_dim)，也就是(3, 4)
# 对提取出的第一个头的矩阵进行矩阵乘法操作，这里是与它自身的转置相乘
# 这种操作在很多场景下（比如在计算注意力机制中的自注意力分数时）是常见的步骤
# 相乘的结果first_res的形状为(num_tokens, num_tokens)，即(3, 3)
first_res = first_head @ first_head.T
print("First head:\n", first_res)
# 同样地，提取出张量a中的第二个头（索引为1）和第一个样本（索引为0）的数据部分，即第二个头对应的矩阵
second_head = a[0, 1, :, :]
# 此时second_head的形状也为(num_tokens, head_dim)，同样是(3, 4)
# 对提取出的第二个头的矩阵进行矩阵乘法操作，与它自身的转置相乘
# 相乘的结果second_res的形状为(num_tokens, num_tokens)，即(3, 3)
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# Summary and takeaways

- See the [./multihead-attention.ipynb](./multihead-attention.ipynb) code notebook, which is a concise version of the data loader (chapter 2) plus the multi-head attention class that we implemented in this chapter and will need for training the GPT model in upcoming chapters
- You can find the exercise solutions in [./exercise-solutions.ipynb](./exercise-solutions.ipynb)